## Install
vnpy_fxdayu: https://github.com/xingetouzi/vnpy_fxdayu

Mongodb: https://www.mongodb.com/download-center#community

安装说明: https://github.com/xingetouzi/vnpy_fxdayu/wiki/Windows环境安装

建议安装robomongo作为可视化数据库管理

## 根据csv数据格式入数据
1. pymongo
2. VT_setting.json

数据下载地址： https://pan.baidu.com/s/1BiIjSX2dCYPy03ZcC7NR5A

In [1]:
import csv
from datetime import datetime, timedelta
from time import time
import pymongo
from vnpy.trader.vtGlobal import globalSetting
from vnpy.trader.vtConstant import *
from vnpy.trader.vtObject import VtBarData

def loadCoinCsv(fileName, dbName, symbol):
    """将数字货币数据导出的csv格式的历史分钟数据插入到Mongo数据库中"""
    start = time()
    print('开始读取CSV文件%s中的数据插入到%s的%s中' %(fileName, dbName, symbol))

    # 锁定集合，并创建索引
    client = pymongo.MongoClient(globalSetting['mongoHost'], globalSetting['mongoPort'])
    collection = client[dbName][symbol]
    collection.ensure_index([('datetime', pymongo.ASCENDING)], unique=True)

    # 读取数据和插入到数据库
    reader = csv.reader(open(fileName,"r"))

    for d in reader:
        if len(d[-1]) == 16:
            bar = VtBarData()
            bar.vtSymbol = d[-1]
            bar.symbol, bar.exchange = bar.vtSymbol.split(':')
            bar.datetime = datetime.strptime(d[3], '%Y-%m-%d %H:%M:%S')
            bar.date = bar.datetime.date().strftime('%Y%m%d')
            bar.time = bar.datetime.time().strftime('%H:%M:%S')
            if bar.datetime:
                bar.high = float(d[6])
                bar.low = float(d[7])
                bar.open = float(d[8])
                bar.close = float(d[1])
                bar.volume = float(d[13])

            flt = {'datetime': bar.datetime}
            collection.update_one(flt, {'$set':bar.__dict__}, upsert=True)
            print('%s \t %s' % (bar.date, bar.time))

    print('插入完毕，耗时：%s' % (time()-start))

In [7]:
loadCoinCsv('tBTCUSD.csv', "VnTrader_1Min_Db", 'tBCHUSD:bitfinex')

## 安装datautils可以管理Mongodb的数据
github地址:  https://github.com/cheatm/datautils

In [8]:
from datautils.mongodb import read
from pymongo import MongoClient
from datetime import datetime

client = MongoClient("192.168.0.104", 37017)
data = read(client["VnTrader_1Min_Db"]["tBCHUSD:bitfinex"], datetime=(datetime(2018, 6, 29 ),datetime(2018, 6, 30 )))
print(data)

       close      date            datetime  exchange gatewayName    high  \
0     696.57  20180629 2018-06-29 00:00:00  bitfinex              696.57   
1     696.29  20180629 2018-06-29 00:01:00  bitfinex              696.29   
2     695.70  20180629 2018-06-29 00:02:00  bitfinex              695.70   
3     696.22  20180629 2018-06-29 00:05:00  bitfinex              696.22   
4     696.26  20180629 2018-06-29 00:06:00  bitfinex              696.26   
5     696.25  20180629 2018-06-29 00:07:00  bitfinex              696.26   
6     696.26  20180629 2018-06-29 00:09:00  bitfinex              696.26   
7     696.91  20180629 2018-06-29 00:11:00  bitfinex              696.91   
8     697.47  20180629 2018-06-29 00:12:00  bitfinex              697.47   
9     697.46  20180629 2018-06-29 00:13:00  bitfinex              697.46   
10    697.41  20180629 2018-06-29 00:14:00  bitfinex              697.41   
11    697.40  20180629 2018-06-29 00:15:00  bitfinex              697.42   
12    697.37